In [2]:
import os
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
# neural network modules
import torch.nn as nn
# optimizers
import torch.optim as optim
# transformations 
import torchvision.transforms as transforms

from torchvision import models
import torchvision
from torch.utils.data import DataLoader, Dataset, Subset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Download dataset 
https://drive.google.com/drive/folders/1vI8Bkk5DojitLNkpz-UT30jOEay0XXon?usp=sharing

## Load data

In [3]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
y_train = y_train.squeeze()

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")
y_test = y_test.squeeze()

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


In [5]:
# hypterparameters
num_epochs = 5
batch_size = 64
lr = 0.005

# calculate the mean and std of the cifar10 dataset
def get_mean_and_std(x_train):
    mean = []
    std = []
    for i in range(3):
        mean.append(np.mean((x_train/255)[:, :, :, i]))
        std.append(np.std((x_train/255)[:, :, :, i]))

    mean = np.array(mean)
    std = np.array(std)

    return mean, std

class Cifar10(Dataset):
    def __init__(self, data, label, transform=None):
        self.data = data
        self.label = label
        self.transform = transform

    def __getitem__(self, index):
        img = self.data[index]
        if self.transform:
            img = self.transform(img)
        return img, self.label[index]

    def __len__(self):
        return len(self.data)

# data augmentation with resize and normalization (project data to [-1, 1])
mean, std = get_mean_and_std(x_train)
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),        
        transforms.Normalize(mean, std)
    ])

train_dataset = Cifar10(x_train, y_train, transform=transform)
test_dataset = Cifar10(x_test, y_test, transform=transform)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
dataiter = iter(test_dataloader)
images, labels = dataiter.next()
assert images.shape[0] == batch_size

# Model Reference
[DenseNet (CVPR 2017)](https://arxiv.org/pdf/1608.06993.pdf)  
[FINETUNING TORCHVISION MODELS](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)

In [7]:
################################ FINETUNING ##################################

# Load a pretrained model and reset final fully connected layer

# load a pretrained model (densenet121)
model = models.densenet121(pretrained=True)

# reset final fully connected layer (num_ftrs = 1024)
num_ftrs = model.classifier.in_features

model.classifier = nn.Sequential(
                        nn.Linear(num_ftrs, 256),  
                        nn.ReLU(), 
                        nn.Dropout(0.2),
                        nn.Linear(256, 10))

# copy weights for futher retraining on full train dataset
model_wts = copy.deepcopy(model.state_dict())

# move model to a device
model = model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=lr)

In [8]:
def train(model, criterion, optimizer, num_epochs, dataloader_train):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):

        # set model to the training mode
        model.train()

        n_samples = 0
        correct_train = 0
        epoch_loss_train = 0

        loop = tqdm(enumerate(dataloader_train), total=len(dataloader_train))
        for i, (images, labels) in loop:
            
            images = images.to(device)
            labels = labels.to(device)

            # forward pass (pred)
            pred = model(images)

            # loss
            loss = criterion(pred, labels)

            # mean loss * num samples in batch
            epoch_loss_train += labels.shape[0] * loss.item()

            # empty gradients
            optimizer.zero_grad()

            # gradient (backpropagation)
            loss.backward()

            # update weights
            optimizer.step()

            # values, indexes
            value, predicted = torch.max(pred, 1)

            # += batch_size
            n_samples += labels.shape[0]
            # num of correctly predicted in this batch
            correct_train += (predicted == labels).sum().item()

            loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
            loop.set_postfix(loss=epoch_loss_train / n_samples, acc=f'{100 * correct_train / n_samples:.2f}%')

        # train acc per epoch
        train_acc = 100 * correct_train / n_samples
        # train loss per epoch
        epoch_loss_train = epoch_loss_train / n_samples

        print(f'Train accuracy: {train_acc:.2f}, loss: {epoch_loss_train:.2f}')

        # find best accuracy on training data
        if train_acc > best_acc:
            best_acc = train_acc
            # copy current model weights
            best_model_wts = copy.deepcopy(model.state_dict())

        print('=' * 80)

    print('Best train acc: {:2f}'.format(best_acc))

    # load best weights
    model.load_state_dict(best_model_wts)

    return model

In [9]:
model = train(model, criterion, optimizer, num_epochs, train_dataloader)

Epoch [1/5]: 100%|██████████| 782/782 [03:58<00:00,  3.28it/s, acc=76.35%, loss=0.86] 


Train accuracy: 76.35, loss: 0.86


Epoch [2/5]: 100%|██████████| 782/782 [03:42<00:00,  3.51it/s, acc=93.07%, loss=0.22] 


Train accuracy: 93.07, loss: 0.22


Epoch [3/5]: 100%|██████████| 782/782 [03:49<00:00,  3.40it/s, acc=95.92%, loss=0.131]


Train accuracy: 95.92, loss: 0.13


Epoch [4/5]: 100%|██████████| 782/782 [03:49<00:00,  3.41it/s, acc=97.85%, loss=0.077] 


Train accuracy: 97.85, loss: 0.08


Epoch [5/5]: 100%|██████████| 782/782 [03:43<00:00,  3.51it/s, acc=99.03%, loss=0.0429]

Train accuracy: 99.03, loss: 0.04
Best train acc: 99.030000


In [10]:
################################### PREDICT ####################################

def model_predict(model, test_dataloader):
    predictions = np.array([])

    with torch.no_grad():
        model.eval()

        for images, _ in test_dataloader:
            images = images.to(device)

            outputs = model(images)

            # value, index
            v, pred = torch.max(outputs, 1)

            pred = pred.cpu().numpy()

            predictions = np.concatenate((predictions, pred), axis=None)

    return predictions

In [11]:
# save model
torch.save(model.state_dict(), 'HW5_weights.pth')

![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [12]:
y_pred = model_predict(model, test_dataloader)

In [13]:
assert y_pred.shape == (10000,)

In [14]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.9562
